In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
import talib as ta
from matplotlib.animation import FuncAnimation
import pytz

/var/folders/jj/jptb3v7j2fbbv3f9g55j7fc40000gn/T/ipykernel_22852/1279198250.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/jj/jptb3v7j2fbbv3f9g55j7fc40000gn/T/ipykernel_22852/1279198250.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
class MovingAverageDayTrading():

    def __init__(self, ticker, stop_loss=0.05, take_profit=0.1):
        self.ticker = ticker
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        # This way, when an instance of the class is created, the self.stop_loss and self.take_profit will be set with default values of 0.05 and 0.1 respectively. These values can be overridden when an instance of the class is created by passing a different value for stop_loss and take_profit.

    def moving_average_timeframes(self):
        
        #  QUERYING DATA --------------------------------------------

        end= datetime.today()
        start = end - timedelta(days=2)
        df = yf.download(self.ticker, start= start ,end= end, interval='1m')
        print(f'--------{df}__________')
        print(df.dtypes)
        print(f'index typeeeee : {df.index.dtype}')
        
        # check the interval of your dataframe by using the df.resample function with the same interval of data you want to work with
        ## df.resample('1min')

        #  SETTING UP DATA: ADDING COLUMNS FOR MA  --------------------------------------

        # Exponential Moving Averages
        df['EMA_20'] = df['Adj Close'].ewm(span=20, adjust=False).mean()
        df['EMA_50'] = df['Adj Close'].ewm(span=50, adjust=False).mean()
        df['EMA_200'] = df['Adj Close'].ewm(span=200, adjust=False).mean()
        # the ewm() method to calculate the EMA, passing in the window size or span, which is the number of periods to use in the moving average. You can also adjust whether or not to apply some decay to the weights by adjusting the adjust parameter. By default is set to True, but it depends on the use case.
        
        
        # CLEANING DATA ------------------------------------------

        print(f'@@@@@@@@@@{df.columns}@@@@@@@@@@')
        print(df.dtypes)

        # check if the data is sorted correctly by running 
        df.sort_index(inplace=True)

        # Remove any rows with missing values
        df = df.dropna()
        # fill the empty spaces with what we want to fill it out
        df.fillna(method='ffill', inplace=True)

        # convert the index to the correct timezone using the pytz library
        # df.index = df.index.tz_localize('UTC').tz_convert('US/Eastern')

        df = df.between_time('09:30', '16:00', axis=0)
        
        # Filter the data to only include rows within trading hours
        df = df[(df.index.hour >= 9) & (df.index.hour <= 16)]
        # Remove any rows for weekends (Saturday and Sunday)
        df = df[df.index.weekday < 5]
        # Select only the columns of interest
        df = df[['Adj Close', 'EMA_20', 'EMA_50', 'EMA_200']]#,'SMA_20', 'SMA_50', 'SMA_200', 'BOL_upper','STO_slowk', 'RSI', 'MACD']]

        print(f'SHAPEEEEEEEEEEE  {df.shape}')

        
        if self.stop_loss is None or self.take_profit is None:
            raise ValueError("stop_loss and take_profit values must be set before using them")

        #  RISK MANAGEMENT // CHECKS BEFORE PLOTTING ----------------------------

        # buy = []
        # sell = []
        # for i in range(1, len(df)):
        #     # Check for buy signal
        #     if (df['EMA_20'][i] > df['EMA_50'][i] and df['EMA_20'][i-1] < df['EMA_50'][i-1]) and (df['EMA_20'][i] > df['EMA_200'][i] and df['EMA_20'][i-1] < df['EMA_200'][i-1]) and (df['SMA_20'][i] > df['SMA_50'][i] and df['SMA_20'][i-1] < df['SMA_50'][i-1]) and (df['SMA_20'][i] > df['SMA_200'][i] and df['SMA_20'][i-1] < df['SMA_200'][i-1]):
        #         buy.append((df.index[i], df['Adj Close'][i]))
        #         current_price = df['Adj Close'][i]
        #         stop_loss_price = current_price * (1 - self.stop_loss)
        #         take_profit_price = current_price * (1 + self.take_profit)
                
        #         # Implement stop loss and take profit
        #         for j in range(i, len(df)):
        #             if df['Adj Close'][j] < stop_loss_price:
        #                 sell.append((df.index[j], df['Adj Close'][j]))
        #                 break
        #             elif df['Adj Close'][j] > take_profit_price:
        #                 sell.append((df.index[j], df['Adj Close'][j]))
        #                 break
        #     # Check for sell signal   
        #     elif (df['EMA_20'][i] < df['EMA_50'][i] and df['EMA_20'][i-1] > df['EMA_50'][i-1]) and (df['EMA_20'][i] < df['EMA_200'][i] and df['EMA_20'][i-1] > df['EMA_200'][i-1]) and (df['SMA_20'][i] < df['SMA_50'][i] and df['SMA_20'][i-1] > df['SMA_50'][i-1]) and (df['SMA_20'][i] < df['SMA_200'][i] and df['SMA_20'][i-1] > df['SMA_200'][i-1]):
        #         sell.append((df.index[i], df['Adj Close'][i]))

        # PLOTTING WITH MATPLOTLIB -------------------------------------
        
        app = dash.Dash()

        chart = px.line(df, x='Adj Close', y=['Adj Close', 'EMA_20', 'EMA_50', 'EMA_200'], title='Moving Average Day Trading')

        chart.update_layout(title='Moving Average Day Trading', xaxis_title='Date', yaxis_title='Price')

        # Add the chart to the Dash app
        app.layout = html.Div([
            dcc.Graph(figure=chart)
        ])

        return app.run_server(debug=True)

        # df = pd.DataFrame(df.values.reshape(181,))

        # df = df.values.ravel()

if __name__ == '__main__':
    
    average = MovingAverageDayTrading('GOOG', stop_loss=0.03, take_profit=0.15)
    print(average.moving_average_timeframes())

[*********************100%***********************]  1 of 1 completed
--------                          Open       High        Low      Close  Adj Close  \
Datetime                                                                     
2023-01-10 15:02:00  88.735001  88.779999  88.680000  88.730003  88.730003   
2023-01-10 15:03:00  88.730003  88.750000  88.680000  88.750000  88.750000   
2023-01-10 15:04:00  88.742401  88.800003  88.739998  88.769997  88.769997   
2023-01-10 15:05:00  88.769997  88.809998  88.760002  88.760002  88.760002   
2023-01-10 15:06:00  88.764999  88.790001  88.650002  88.669998  88.669998   
...                        ...        ...        ...        ...        ...   
2023-01-11 15:56:00  92.245003  92.260002  92.180000  92.220001  92.220001   
2023-01-11 15:57:00  92.220001  92.239998  92.199997  92.214996  92.214996   
2023-01-11 15:58:00  92.209999  92.269997  92.199997  92.250000  92.250000   
2023-01-11 15:59:00  92.245003  92.320000  92.199997  92.239998  

/Users/andresfelipecastrosalazar/Desktop/Pythonplatzy/nmpy/venv/lib/python3.10/site-packages/traitlets/traitlets.py:2412: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/Users/andresfelipecastrosalazar/Desktop/Pythonplatzy/nmpy/venv/lib/python3.10/site-packages/traitlets/traitlets.py:2366: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use 'ad7e7721-9ff3-4d28-b040-eca0b936ce49' instead of 'b"ad7e7721-9ff3-4d28-b040-eca0b936ce49"'.
  warn(
Traceback (most recent call last):
  File "/Users/andresfelipecastrosalazar/Desktop/Pythonplatzy/nmpy/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/andresfelipecastrosalazar/Desktop/Pythonplatzy/nmpy/venv/lib/python3.10/site-packages/traitlets/config/application.py", line 981, in launch_instance
    app.initial

SystemExit: 1

/Users/andresfelipecastrosalazar/Desktop/Pythonplatzy/nmpy/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

